In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url='https://www.imdb.com/chart/top/'

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
response=requests.get(url,headers=headers)